# Mask R-CNN (Wild Dash)

**(Esegui tutto)**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install pyyaml==5.1

from IPython.display import Image, clear_output

# Cerchiamo la versione che porti il match perfetto tra le librerie
import torch
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]

!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/$CUDA_VERSION/torch$TORCH_VERSION/index.html

clear_output()

In [3]:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# Common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# Common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

**Decommentare modello che si vuole usare**

In [4]:
ResNet_101_FPN = "COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml"
# ResNet_101_C4 = "COCO-InstanceSegmentation/mask_rcnn_R_101_C4_3x.yaml"
# ResNet_101_DC5 = "COCO-InstanceSegmentation/mask_rcnn_R_101_DC5_3x.yaml"
# Resnet_50_FPN = "COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_1x.yaml"
# Resnet_50_C4 = "COCO-InstanceSegmentation/mask_rcnn_R_50_C4_1x.yaml"
# Resnet_50_DC5 = "COCO-InstanceSegmentation/mask_rcnn_R_50_DC5_1x.yaml"

# ecc ecc 


In [5]:
from detectron2.data.datasets import register_coco_instances
register_coco_instances("dash_train", {}, "/content/drive/MyDrive/datasets/wd_public_02/wilddash_train.json", "/content/drive/MyDrive/datasets/wd_public_02/images_train")


from detectron2.data.datasets import register_coco_instances
register_coco_instances("dash_test", {}, "/content/drive/MyDrive/datasets/wd_public_02/wilddash_test.json", "/content/drive/MyDrive/datasets/wd_public_02/images_test")

**Cambiare qui i parametri**

In [ ]:
from detectron2.engine import DefaultTrainer

# CFG OBBLIGATORIE
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file(ResNet_101_FPN))
cfg.DATASETS.TRAIN = ("dash_train",)
cfg.DATASETS.TEST = ()
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(ResNet_101_FPN)
cfg.INPUT.MASK_FORMAT = 'bitmask'


#CFG SETTABILI
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128 # default 512 - numero di img per calcolare la loss ad ogni step
#cfg.DATALOADER.NUM_WORKERS = 2
cfg.SOLVER.IMS_PER_BATCH = 1 # default 16 (# Number of images per batch across all machines. This is also the number
                                          # of training images per step (i.e. per iteration). If we use 16 GPUs
                                          # and IMS_PER_BATCH = 32, each GPU will see 2 images per batch.)
cfg.SOLVER.BASE_LR = 0.001 # default 0.001
cfg.SOLVER.MAX_ITER = 4000 # default 40.000 (provare 4000)
cfg.SOLVER.STEPS = (3000,) # default 30.000 (provare 300) (The iteration number to decrease learning rate by GAMMA. GAMMA = 0.1 default)




# ESPERIMENTI FREEZE

# Freeze the first several stages so they are not trained.
# There are 5 stages in ResNet. The first is a convolution, and the following
# stages are each group of residual blocks.
#cfg.MODEL.BACKBONE.FREEZE_AT = 2 #default 2


# Classi di Cityscapes
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 13


os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

[02/03 10:51:29 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
model_final_a3ec72.pkl: 254MB [00:04, 52.8MB/s]                           
Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (14, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (14,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor

[02/03 10:51:41 d2.engine.train_loop]: Starting training from iteration 0


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/detectron2/data/detection_utils.py:433: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ../torch/csrc/utils/tensor_numpy.cpp:189.)
  torch.stack([

[02/03 10:52:09 d2.utils.events]:  eta: 1:17:51  iter: 19  total_loss: 4.462  loss_cls: 2.68  loss_box_reg: 0.8793  loss_mask: 0.6926  loss_rpn_cls: 0.06689  loss_rpn_loc: 0.2447  time: 1.1185  data_time: 0.2577  lr: 1.9981e-05  max_mem: 2228M
[02/03 10:52:31 d2.utils.events]:  eta: 1:13:31  iter: 39  total_loss: 4.115  loss_cls: 2.281  loss_box_reg: 0.8436  loss_mask: 0.6893  loss_rpn_cls: 0.02953  loss_rpn_loc: 0.1001  time: 1.1036  data_time: 0.0071  lr: 3.9961e-05  max_mem: 2229M
[02/03 10:52:53 d2.utils.events]:  eta: 1:15:34  iter: 59  total_loss: 3.257  loss_cls: 1.561  loss_box_reg: 0.7429  loss_mask: 0.6674  loss_rpn_cls: 0.02886  loss_rpn_loc: 0.2223  time: 1.1092  data_time: 0.0060  lr: 5.9941e-05  max_mem: 2229M
[02/03 10:53:15 d2.utils.events]:  eta: 1:13:46  iter: 79  total_loss: 2.575  loss_cls: 0.9123  loss_box_reg: 0.8189  loss_mask: 0.614  loss_rpn_cls: 0.02853  loss_rpn_loc: 0.0668  time: 1.1054  data_time: 0.0057  lr: 7.9921e-05  max_mem: 2229M
[02/03 10:53:38 d2.ut

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
predictor = DefaultPredictor(cfg)

In [ ]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader

evaluator = COCOEvaluator("dash_test", cfg, False, output_dir="./output")
val_loader = build_detection_test_loader(cfg, "dash_test")
print(inference_on_dataset(predictor.model, val_loader, evaluator))

**Stampa immagini**

In [ ]:
def print_inference(img_path, predictor):
  img = cv2.imread(img_path)
  outputs = predictor(img)
  v = Visualizer(img[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=0.5)
  out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
  cv2_imshow(out.get_image()[:, :, ::-1])


# Esempio di utilizzo
print_inference("/content/drive/MyDrive/datasets/wd_public_02/images_test/tk0000_100000.jpg", predictor)

In [ ]:
%load_ext tensorboard
%tensorboard --logdir output